In [ ]:
import os
import zipfile

local_zip = 'wayang-new.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('.')
zip_ref.close()

In [1]:
base_dir = './new-reduced/'

wayang_classes = os.listdir(base_dir)
print(wayang_classes)

['bagong', 'gareng', 'petruk', 'semar']


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
image_batch_size = 20
aug_image_total = 40

new_datagen = ImageDataGenerator(rescale = 1.0/255,
                                 horizontal_flip = True,
                                #  width_shift_range = 0.1,
                                #  height_shift_range = 0.1,
                                 fill_mode = 'nearest',
                                 rotation_range=15,
                                 zoom_range=0.2)

In [4]:
augmented_dir = './augmented-reduced'

In [6]:
for wayang_class in wayang_classes:
    aug_image_step = 0
    new_generator = new_datagen.flow_from_directory(base_dir, batch_size=image_batch_size,
                                                    class_mode='binary', classes=[wayang_class],
                                                    target_size=(225, 175),
                                                    shuffle=False, save_to_dir='/'.join([augmented_dir, wayang_class]),
                                                    save_prefix='', save_format='jpg')
    for image_batch in new_generator:
        aug_image_step += 1
        if aug_image_step > aug_image_total:
            break

Found 46 images belonging to 1 classes.
Found 45 images belonging to 1 classes.
Found 44 images belonging to 1 classes.
Found 48 images belonging to 1 classes.


In [8]:
new_generator = new_datagen.flow_from_directory(augmented_dir, batch_size=image_batch_size,
                                                    class_mode='categorical',
                                                    target_size=(225, 175),
                                                    )

labels_dict = new_generator.class_indices
wayang_labels = dict((v, k) for k, v in labels_dict.items())
print(wayang_labels)

Found 2539 images belonging to 4 classes.
{0: 'bagong', 1: 'gareng', 2: 'petruk', 3: 'semar'}


In [9]:
import sklearn
import numpy as np

class_weight_list = sklearn.utils.class_weight.compute_class_weight('balanced',
                                                               np.unique(new_generator.classes),
                                                               new_generator.classes)

class_weights = {}

for k, _ in wayang_labels.items():
  class_weights[k] = class_weight_list[k]

print(class_weights)

{0: 0.9949059561128527, 1: 1.0156, 2: 1.0371732026143792, 3: 0.9559487951807228}
C:\Python38\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3], y=[0 0 0 ... 3 3 3] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [44]:
train_batch = 20
validation_batch = 20

In [46]:
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                 horizontal_flip = True,
                                 width_shift_range = 0.1,
                                 height_shift_range = 0.1,
                                 fill_mode = 'nearest',
                                 rotation_range=15,
                                 zoom_range=0.2,
                                 validation_split=0.2)

train_generator = train_datagen.flow_from_directory(augmented_dir, batch_size=train_batch,
                                                    class_mode='categorical', target_size=(225, 175),
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(augmented_dir, batch_size=validation_batch,
                                                         class_mode='categorical', target_size=(225, 175),
                                                         subset='validation')

Found 3575 images belonging to 4 classes.
Found 893 images belonging to 4 classes.


In [7]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'